# Speech Denosing using RNN # 

This problem uses RNN as a tool for Speech Denoising purpose.

In [1]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="3"

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import librosa
from tensorflow.contrib import rnn
import glob
import pickle


## Preprocessing of Training Set ##



In [5]:
############ LOADING OF TRAINING DATA #################

train = []
signal = []
noise = []
X_list = []

import os, re
for f in sorted(glob.glob("/opt/e533/timit-homework/tr/trx*")):
        st, sr=librosa.load(f, sr=None)
        X=librosa.stft(st, n_fft=1024, hop_length=512)
        X_abs = np.abs(X)
        train.append(np.transpose(X_abs))
        X_list.append(np.transpose(X))
        
for f in sorted(glob.glob("/opt/e533/timit-homework/tr/trn*")):
        sn, sr=librosa.load(f, sr=None)
        X=librosa.stft(sn, n_fft=1024, hop_length=512)
        X_abs = np.abs(X)
        noise.append(np.transpose(X_abs))
        
for f in sorted(glob.glob("/opt/e533/timit-homework/tr/trs*")):
        sc, sr=librosa.load(f, sr=None)
        X=librosa.stft(sc, n_fft=1024, hop_length=512)
        X_abs = np.abs(X)
        signal.append(np.transpose(X_abs))

In [6]:
############### SAVING THE FILES AS PICKLE ############
# with open('train.pkl', 'wb') as f:
#      pickle.dump(train, f)
    
# with open('noise.pkl', 'wb') as f:
#      pickle.dump(noise, f)
    
# with open('signal.pkl', 'wb') as f:
#      pickle.dump(signal, f)
        
# with open('X_list.pkl', 'wb') as f:
#      pickle.dump(X_list, f)        

In [2]:
################ LOADING OF PICKLE FILES ############

with open('train.pkl', 'rb') as f:
    train1 = pickle.load(f)
    
with open('noise.pkl', 'rb') as f:
    noise1 = pickle.load(f)
    
with open('signal.pkl', 'rb') as f:
    signal1 = pickle.load(f)
    
with open('X_list.pkl', 'rb') as f:
    X_list = pickle.load(f)

In [3]:
############# PADDING WITH ZEROS ##########
train_fin = train1
noise_fin = noise1
signal_fin = signal1
X_fin = X_list

# for i in range(len(train1)):
#     z = np.zeros((178-len(train1[i]), 513))
#     a  = np.concatenate((train1[i], z),axis = 0)
#     train_fin.append(a)
    
#     z = np.zeros((178-len(signal1[i]), 513))
#     a  = np.concatenate((signal1[i], z),axis = 0)
#     signal_fin.append(a)
    
#     z = np.zeros((178-len(noise1[i]), 513))
#     a  = np.concatenate((noise1[i], z),axis = 0)
#     noise_fin.append(a)
    
    
#     z = np.zeros((178-len(X_list[i]), 513),dtype = 'complex64')
#     a  = np.concatenate((X_list[i], z),axis = 0)
#     X_fin.append(a)

In [4]:
################## IBM CREATION #######################
ibm = []
for i in range(len(signal_fin)):
    ibm.append(np.greater(signal_fin[i],noise_fin[i])*1)
    
S_hat = []
for i in range(1200):               
    S_hat.append(np.multiply(ibm[i], X_fin[i]))

## Preprocessing of Validation Set 

In [ ]:
############ LOADING THE VALIDATION AUDIO CLIPS ###############

train_val = []
signal_val = []
noise_val = []
V_list = []
s_clean = []

import os, re
for f in sorted(glob.glob("/opt/e533/timit-homework/v/vx*")):
        st, sr=librosa.load(f, sr=None)
        V=librosa.stft(st, n_fft=1024, hop_length=512)
        V_abs = np.abs(V)
        train_val.append(np.transpose(V_abs))
        V_list.append(np.transpose(V))
        
for f in sorted(glob.glob("/opt/e533/timit-homework/v/vn*")):
        sn, sr=librosa.load(f, sr=None)
        V=librosa.stft(sn, n_fft=1024, hop_length=512)
        V_abs = np.abs(V)
        noise_val.append(np.transpose(V_abs))
        
for f in sorted(glob.glob("/opt/e533/timit-homework/v/vs*")):
        sc, sr=librosa.load(f, sr=None)
        V=librosa.stft(sc, n_fft=1024, hop_length=512)
        s_clean.append(V)
        V_abs = np.abs(V)
        signal_val.append(np.transpose(V_abs))

In [11]:
############ SAVING THE FILES AS PICKLE #############

# with open('train_val.pkl', 'wb') as f:
#      pickle.dump(train_val, f)
    
# with open('noise_val.pkl', 'wb') as f:
#      pickle.dump(noise_val, f)
    
# with open('signal_val.pkl', 'wb') as f:
#      pickle.dump(signal_val, f)
        
# with open('V_list.pkl', 'wb') as f:
#      pickle.dump(V_list, f) 

# with open('s_clean.pkl', 'wb') as f:
#    pickle.dump(s_clean, f)

In [5]:
############### LOADING THE SAVED PICKLE FILES ###############

with open('train_val.pkl', 'rb') as f:
    train1_val = pickle.load(f)
    
with open('noise_val.pkl', 'rb') as f:
    noise1_val = pickle.load(f)
    
with open('signal_val.pkl', 'rb') as f:
    signal1_val = pickle.load(f)
    
with open('V_list.pkl', 'rb') as f:
    V1_list = pickle.load(f)
    
with open('s_clean.pkl', 'rb') as f:
    s1_clean = pickle.load(f)

In [6]:
############# IBM MATRIX FOR VALIDATION SET ##########

ibm_val = []
for i in range(len(signal_fin)):
    ibm_val.append(np.greater(signal1_val[i],noise1_val[i])*1)

In [7]:
len_arr = [len(x) for x in train1_val]

## Preprocessing  of Testing Data 

In [ ]:
################ LOADING OF THE TEST DATA ##################
test = []
test_complex_list = []

import os, re
for f in sorted(glob.glob("/opt/e533/timit-homework/te/te*")):
        st, sr=librosa.load(f, sr=None)
        X=librosa.stft(st, n_fft=1024, hop_length=512)
        X_abs = np.abs(X)
        test.append(np.transpose(X_abs))
        test_complex_list.append(np.transpose(X))
        

In [15]:
########### SAVING TEST DATA USING PICKLE ############
# with open('test.pkl', 'wb') as f:
#      pickle.dump(test, f)
        
# with open('test_complex.pkl', 'wb') as f:
#      pickle.dump(test_complex_list, f)


In [8]:
############ LOADING PICKLE FILES ##############

with open('test_complex.pkl', 'rb') as f:
    test_comp1 = pickle.load(f)
    
with open('test.pkl', 'rb') as f:
    test1 = pickle.load(f)

In [9]:
len_test = [len(x) for x in test1]

## Graph Definition ##

In [10]:
tf.reset_default_graph()

In [11]:
# Training Parameters
learning_rate = 0.005
dr = 0.9
batch_size = 10
display_step = 1

# Network Parameters
num_input = 513 # MNIST data input (img shape: 28*28)
num_layers = 2

num_hidden = 513 
num_classes = 513 

# Input
X = tf.placeholder("float", [None,None, num_input])
Y = tf.placeholder("float", [None,None, num_classes])
dropout = tf.placeholder(tf.float32)

Here, a 2-layered RNN is used for the purpose of denoising. Each cell is wrapped in a Dropout layer for the purpose of avoiding overfitting. 

In [12]:
############ RNN CELL STRUCTURE #################

def RNN(X):

    lstm_cells = []
    
    #Addition of Layers
    for i in range(num_layers):
    
    # Define a lstm cell with tensorflow and wrapping  it in Dropout 
        lstm_cell = rnn.LSTMCell(num_hidden, forget_bias=1.0)
        lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=dropout)
        lstm_cells.append(lstm_cell)
        
    lstm_cell = rnn.MultiRNNCell(lstm_cells)
    
    # Get lstm cell output
    outputs, states = tf.nn.dynamic_rnn(lstm_cell, X, dtype=tf.float32)

    return outputs, states

In [13]:
################# LOSS FUNCTION AND OPTIMIZER DEFINITION ###################

output, state = RNN(X)

# Defining logits and output 

logits = tf.contrib.layers.fully_connected(output, num_classes, activation_fn=None)
prediction = logits

flat_Y = tf.reshape(Y, [-1] + Y.shape.as_list()[2:])
flat_logit = tf.reshape(logits, [-1] + logits.shape.as_list()[2:])



# Defining loss and optimizer
loss_op =tf.losses.mean_squared_error(predictions=flat_logit, labels=flat_Y)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

#Defining accuracy measure 
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

print(flat_Y.shape)
print(flat_logit.shape)

(?, 513)
(?, 513)


In [14]:
############################# TRAINING OF THE NETWORK ##################################
gpu_options = tf.GPUOptions(allow_growth=True)
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess: 

    # Run the initializer
    sess.run(tf.global_variables_initializer())
    
    #Training for 31 epochs 
    for epoch in range(31):
        ep_loss = 0
        for i in range(0, 1200,10):
            batch_target = np.array(ibm[i:i+10])
            batch_data = np.array(train_fin[i:i+10])
            c,_ = sess.run([loss_op, optimizer], {X: batch_data, Y: batch_target, dropout: dr})
            
            ep_loss += c
        print('Loss at epoch',epoch+1, 'is: ', ep_loss)

    print("Optimization Finished!")
    
    # Using the model for Prediction of the Validation and Testing Sets
    pred = []
    pred_test = []
    
    
    # Validation evaluation
    print('Starting Evaluation')
    for i in range(1200):
        pred.append(sess.run(prediction, {X: np.reshape(V1_list[i],(-1,len_arr[i],513)), Y: np.reshape(ibm_val[i],(-1,len_arr[i],513)), dropout: dr} ))
        
    print('Evaluation completed')
    
    
    # Testing of Data
    print('Testing Set Evaluation Started')
    for i in range(len(test1)):
        pred_test.append(sess.run(prediction, {X: np.reshape(test_comp1[i],(-1,len_test[i],513)), dropout: dr} ))
    
    print('Testing Set Evaluation Completed!')

Loss at epoch 1 is:  27.7763442248106
Loss at epoch 2 is:  21.63682208955288
Loss at epoch 3 is:  20.0035180747509
Loss at epoch 4 is:  19.092994891107082
Loss at epoch 5 is:  18.468937255442142
Loss at epoch 6 is:  17.980053022503853
Loss at epoch 7 is:  17.5129998549819
Loss at epoch 8 is:  17.203936092555523
Loss at epoch 9 is:  16.8566512465477
Loss at epoch 10 is:  16.567163921892643
Loss at epoch 11 is:  16.312053076922894
Loss at epoch 12 is:  16.100208692252636
Loss at epoch 13 is:  15.836823277175426
Loss at epoch 14 is:  15.584485962986946
Loss at epoch 15 is:  15.501247107982635
Loss at epoch 16 is:  15.33034285157919
Loss at epoch 17 is:  15.058369942009449
Loss at epoch 18 is:  14.870569966733456
Loss at epoch 19 is:  14.705615170300007
Loss at epoch 20 is:  14.557749480009079
Loss at epoch 21 is:  14.551204204559326
Loss at epoch 22 is:  14.472444474697113
Loss at epoch 23 is:  14.240056686103344
Loss at epoch 24 is:  14.086849249899387
Loss at epoch 25 is:  13.9424181431

/opt/Anaconda3-5.0.1/envs/e533/lib/python3.6/site-packages/numpy/core/numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


Evaluation completed
Testing Set Evaluation Started
Testing Set Evaluation Completed!


## Reconstruction of the wave using the predicted Mask ##

In [15]:
############# CREATING S-HAT ##################
M_hat = []
M_list = []
for i in range(1200):
    M_hat.append(np.multiply(pred[i],V1_list[i]))
    M_hat[i] = np.reshape(M_hat[i], (len_arr[i],513))
    M_hat[i] = np.transpose(M_hat[i])

In [16]:
################## WRITING THE NEW WAVE AS A .WAV FILE ########################
s_hat = []

for i in range(1200):
    stest = librosa.istft(M_hat[i], hop_length = 512)
    librosa.output.write_wav(('val_'+str(i)+'.wav'), stest,sr = 16000)
    s_hat.append(stest)

## SNR Calculation ##

In [17]:
snr = []

for i in range(1200):
    isclean =librosa.istft(s1_clean[i],hop_length = 512)

    den = (isclean-s_hat[i])**2
    num = (isclean)**2
    
    snr.append(10*(np.log10(np.sum(num)/np.sum(den))))

print('Max SNR=', max(snr))
print('Average SNR=', np.mean(snr))
print('Min SNR=', min(snr))

Max SNR= 14.493186473846436
Average SNR= 1.5729016558781344
Min SNR= -11.681849956512451


## Reconstruction of Testing Set 

In [18]:
M_test = []

for i in range(400):
    M_test.append(np.multiply(pred_test[i],test_comp1[i]))
    M_test[i] = np.reshape(M_test[i], (len_test[i],513))
    M_test[i] = np.transpose(M_test[i])

In [19]:
s_hat = []

for i in range(400):
    stest = librosa.istft(M_test[i], hop_length = 512)
    librosa.output.write_wav(('test_'+str(i)+'.wav'), stest,sr = 16000)

## References ##

1. https://danijar.com/introduction-to-recurrent-networks-in-tensorflow/

2. https://github.com/aymericdamien/TensorFlow-Examples

3. http://colah.github.io/posts/2015-08-Understanding-LSTMs/
